In [64]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


* @file NLP基礎/NLP_part3(補充).ipynb
  * @brief NLP基礎 模型實作 

  * 此份程式碼是以教學為目的，附有完整的架構解說。

  * @author 人工智慧科技基金會 AI 工程師 - 康文瑋
  * Email: run963741@aif.tw
  * Resume: https://www.cakeresume.com/run963741

  * 最後更新日期: 2020/11/26

# 載入套件

In [65]:
import os
import pandas as pd
import re
import tqdm

from collections import Counter, defaultdict, OrderedDict
from nltk import ngrams

os.chdir('/content/drive/Shared drives/類技術班教材/標準版/NLP基礎')

# 資料前處理

In [66]:
htl = pd.read_csv('Data/ChnSentiCorp_htl_all.csv')

In [67]:
htl.head()

,label,review
0,1,"距離川沙公路較近,但是公交指示不對,如果是""蔡陸線""的話,會非常麻煩.建議用別的路線.房間較..."
1,1,商務大牀房，房間很大，牀有2M寬，整體感覺經濟實惠不錯!
2,1,早餐太差，無論去多少人，那邊也不加食品的。酒店應該重視一下這個問題了。房間本身很好。
3,1,賓館在小街道上，不大好找，但還好北京熱心同胞很多~賓館設施跟介紹的差不多，房間很小，確實挺小...
4,1,"CBD中心,周圍沒什麼店鋪,說5星有點勉強.不知道爲什麼衛生間沒有電吹風"


## 過濾特殊字元

In [68]:
# 正則表達式, https://atedev.wordpress.com/2007/11/23/%E6%AD%A3%E8%A6%8F%E8%A1%A8%E7%A4%BA%E5%BC%8F-regular-expression/
def preprocess(sentence):
  sentence = sentence.strip()
  sentence = re.sub('\s','',sentence)
  # sentence = sentence.replace(u'\u3000',u' ').replace(' ','')
  return list(sentence)

In [69]:
sents = list(htl['review'].apply(preprocess))

In [70]:
sents[0]

['距',
 '離',
 '川',
 '沙',
 '公',
 '路',
 '較',
 '近',
 ',',
 '但',
 '是',
 '公',
 '交',
 '指',
 '示',
 '不',
 '對',
 ',',
 '如',
 '果',
 '是',
 '"',
 '蔡',
 '陸',
 '線',
 '"',
 '的',
 '話',
 ',',
 '會',
 '非',
 '常',
 '麻',
 '煩',
 '.',
 '建',
 '議',
 '用',
 '別',
 '的',
 '路',
 '線',
 '.',
 '房',
 '間',
 '較',
 '爲',
 '簡',
 '單',
 '.']

# N 元語法 (N-gram)

在自然語言處理中有一種任務叫做**文本生成 (Text generation)**，白話來說就是輸入前文給模型，接著預測後文，N 元語法的特性就適合此種任務，N 元語法是透過條件機率來計算，在前面已經出現 N-1 個詞的前提下，計算第 N 個詞的機率，數學式可以表達為：

$$
p(w_N|w_1,\cdots,w_{N-1})=\frac{p(w_N,w_1,\cdots,w_{N-1})}{p(w_1,\cdots,w_{N-1})}
$$

如果要完整預測一個句子 $w_1,\cdots,w_n$，則可以將上式改寫為下式：

$$
p(w_1,\cdots,w_n)=p(w_1)*p(w_2|w_1)*p(w_3|w_1,w_2)*...*p(w_n|w_1,\cdots,w_{n-1})=\prod_{k=1}^np(w_k|w_{0:k-1})
$$

在實際上不可能這樣做，因為每次都要前面所有的詞，計算量太大了，N-gram 的作法就是只考慮前面 N-1 個詞，進而降低計算量，常見的分別為 Bigram 以及 Trigram，分別是考慮前一個詞以及考慮前兩個詞。

* Bigram，每次預測時考慮前一個詞：

$$
p(w_1,\cdots,w_n)=\prod_{k=1}^np(w_k|w_{k-1})
$$

* Trigram，每次預測時考慮前兩個詞：

$$
p(w_1,\cdots,w_n)=\prod_{k=1}^np(w_k|w_{k-2},w_{k-1})
$$

N-gram 的作法用一句話來說明的話，就是**常常一起出現的詞，照理來說在預測時會有高機率一起出現**。


In [71]:
sent = '台灣人工智慧科技基金會'
bigram = ngrams(sent, n=2, pad_left=True, pad_right=True)

for w1, w2 in bigram:
  print('w_n-1: ', w1, '|', 'w_n: ', w2)
  print('-'*25)

w_n-1:  None | w_n:  台
-------------------------
w_n-1:  台 | w_n:  灣
-------------------------
w_n-1:  灣 | w_n:  人
-------------------------
w_n-1:  人 | w_n:  工
-------------------------
w_n-1:  工 | w_n:  智
-------------------------
w_n-1:  智 | w_n:  慧
-------------------------
w_n-1:  慧 | w_n:  科
-------------------------
w_n-1:  科 | w_n:  技
-------------------------
w_n-1:  技 | w_n:  基
-------------------------
w_n-1:  基 | w_n:  金
-------------------------
w_n-1:  金 | w_n:  會
-------------------------
w_n-1:  會 | w_n:  None
-------------------------


In [72]:
sent = '台灣人工智慧科技基金會'
trigram = ngrams(sent, n=3, pad_left=True, pad_right=True)

for w1, w2, w3 in trigram:
  print('w_n-2: ', w1, 'w_n-1: ', w2, '|', 'w_n: ', w3)
  print('-'*35)

w_n-2:  None w_n-1:  None | w_n:  台
-----------------------------------
w_n-2:  None w_n-1:  台 | w_n:  灣
-----------------------------------
w_n-2:  台 w_n-1:  灣 | w_n:  人
-----------------------------------
w_n-2:  灣 w_n-1:  人 | w_n:  工
-----------------------------------
w_n-2:  人 w_n-1:  工 | w_n:  智
-----------------------------------
w_n-2:  工 w_n-1:  智 | w_n:  慧
-----------------------------------
w_n-2:  智 w_n-1:  慧 | w_n:  科
-----------------------------------
w_n-2:  慧 w_n-1:  科 | w_n:  技
-----------------------------------
w_n-2:  科 w_n-1:  技 | w_n:  基
-----------------------------------
w_n-2:  技 w_n-1:  基 | w_n:  金
-----------------------------------
w_n-2:  基 w_n-1:  金 | w_n:  會
-----------------------------------
w_n-2:  金 w_n-1:  會 | w_n:  None
-----------------------------------
w_n-2:  會 w_n-1:  None | w_n:  None
-----------------------------------


# N-gram model

透過上面的範例，我們可以計算出所有詞之間的條件機率，接著就能夠寫出一個簡單的文本生成模型：

In [90]:
class ngram():
  def __init__(self):
    pass
  def __call__(self, n, sents):
    """
    這邊分別實作 bigram 以及 trigram
    """
    model = defaultdict(lambda: defaultdict(lambda: 0))
    if n == 2:
      # 第一組 for 迴圈負責統計所有句子的 n-gram 出現頻率，若第 w2 個詞出現在 w1 詞後面，則次數 +1
      for sent in sents:
        for w1, w2 in ngrams(sent, n, pad_left=True, pad_right=True):
          model[(w1)][w2] += 1

      # 第二組 for 迴圈將曾出現在 w1 後面個別的字(w2)次數除以總次數，得到條件機率
      for w1 in model:
        total_count = float(sum(model[w1].values()))
        for w2 in model[w1]:
          model[w1][w2] /= total_count 

    
    if n == 3:
      for sent in sents:
        for w1, w2, w3 in ngrams(sent, n, pad_left=True, pad_right=True):
          model[(w1,w2)][w3] += 1

      for w1_w2 in model:
        total_count = float(sum(model[w1_w2].values()))
        for w3 in model[w1_w2]:
          model[w1_w2][w3] /= total_count 
    return model

## Bigram

In [ ]:
bigram = ngram()
bigram_model = bigram(2, sents)

In [62]:
# 設定句子最大長度
max_len = 30

# 給予模型起始的字
start_word = '你'

pred_sent = list()
next_word = start_word

for _ in tqdm.tqdm(range(max_len)):
  pred_sent.append(next_word)
  next_word = list(bigram_model[next_word])[0]

pred_sent = ''.join(pred_sent)

100%|██████████| 30/30 [00:00<00:00, 31457.28it/s]


In [63]:
print('Bigram 預測結果: \n', pred_sent)

Bigram 預測結果: 
 你怎麼店應該重視一下這個問題了。酒店應該重視一下這個問題了。


## Trigram

In [91]:
trigram = ngram()
trigram_model = trigram(3, sents)

In [94]:
max_len = 30
start_word = ('缺','點')

pred_sent = list()
next_word = start_word

for _ in tqdm.tqdm(range(max_len)):
  pred_sent += list(next_word)
  input = tuple(pred_sent[-2:])
  next_word = list(trigram_model[input])[0]

pred_sent = ''.join(pred_sent)

100%|██████████| 30/30 [00:00<00:00, 5288.49it/s]


In [95]:
print('Trigram 預測結果: \n', pred_sent)

Trigram 預測結果: 
 缺點：液晶電視固定在牆上，不大好找，但還好北京熱心同胞很多~賓
